In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes transformers
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import pandas as pd
df = pd.read_csv('/content/Psych8k.csv')
df.head()

,Unnamed: 0,instruction,input,output,instructions
0,0,"If you are a counsellor, please answer the que...","Lately, I've been feeling really disconnected ...",Thank you for sharing your experience with me....,NaN
1,1,"If you are a counsellor, please answer the que...","Lately, I've been feeling really stressed and ...",Thank you for sharing your experience with me....,NaN
2,2,"If you are a counsellor, please answer the que...","Lately, I've been feeling a bit off. I sometim...",I appreciate you sharing your concerns with me...,NaN
3,3,"If you are a counsellor, please answer the que...","Lately, I've been feeling a bit off. I've noti...",Thank you for sharing your feelings and experi...,NaN
4,4,"If you are a counsellor, please answer the que...","Lately, I've been feeling quite off. I have be...",I appreciate you sharing your experiences with...,NaN


In [ ]:
df=df.drop(['instructions'],axis=1)
df.head()

,Unnamed: 0,instruction,input,output
0,0,"If you are a counsellor, please answer the que...","Lately, I've been feeling really disconnected ...",Thank you for sharing your experience with me....
1,1,"If you are a counsellor, please answer the que...","Lately, I've been feeling really stressed and ...",Thank you for sharing your experience with me....
2,2,"If you are a counsellor, please answer the que...","Lately, I've been feeling a bit off. I sometim...",I appreciate you sharing your concerns with me...
3,3,"If you are a counsellor, please answer the que...","Lately, I've been feeling a bit off. I've noti...",Thank you for sharing your feelings and experi...
4,4,"If you are a counsellor, please answer the que...","Lately, I've been feeling quite off. I have be...",I appreciate you sharing your experiences with...


In [ ]:
train_df = df.sample(n=7000)
test_df = df.drop(train_df.index)

In [ ]:
from datasets import Dataset
training_data = Dataset.from_pandas(train_df)
testing_data = Dataset.from_pandas(test_df)

In [ ]:
training_data = training_data.remove_columns(['__index_level_0__'])
testing_data = testing_data.remove_columns(['__index_level_0__'])

In [ ]:
print(training_data)

Dataset({
    features: ['Unnamed: 0', 'instruction', 'input', 'output'],
    num_rows: 7000
})


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


In [ ]:
import random

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['input'])):
        text = f'''<s> ### Input: {example['input'][i]}\n ### Response:  {example['output'][i]} ####</s>'''
        output_texts.append(text)

    # Add a new column for the formatted text
    example['formatted_data'] = output_texts
    return example

In [ ]:
def apply_formatting(dataset):
    formatted_dataset = dataset.map(
        formatting_prompts_func,
        batched=True,
        num_proc=8  
    )
    return formatted_dataset

In [ ]:
final_train_dataset = apply_formatting(training_data)
final_test_dataset = apply_formatting(testing_data)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=8):   0%|          | 0/7000 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1187 [00:00<?, ? examples/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 250,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 25,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/7000 [00:00<?, ? examples/s]

In [ ]:
final_train_dataset[0]

{'Unnamed: 0': 1946,
 'instruction': 'If you are a counsellor, please answer the questions based on the description of the patient.',
 'input': "I'm usually successful in handling multiple subjects, but this particular exam just threw me off balance and I failed it.",
 'output': "It sounds like you've been doing well in managing your other subjects and this one exam has been an exception. Even though it's upsetting, it's important to remember that failure is a natural part of life and a learning experience. Can you think of any factors that might have contributed to this unexpected result?",
 'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nIf you are a counsellor, please answer the questions based on the description of the patient.\n\n### Input:\nI'm usually successful in handling multiple subjects, but this particular exam just threw me off balanc

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 250
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
25,1.497600
50,1.199500
75,1.158600
100,1.172200
125,1.149400


Step,Training Loss
25,1.497600
50,1.199500
75,1.158600
100,1.172200
125,1.149400
150,1.140400
175,1.164000
200,1.145400
225,1.159000
250,1.150200


In [19]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "If you are a counsellor, please answer the questions based on the description of the patient.", # instruction
        "I was with a girl one time and we got back to my place. She said she was going to go to the doctor, and everything seemed resolved, but the police were there and I got arrested. I was charged with kidnapping and some other stuff, and I ended up serving some time.", # input
        "", 
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 350)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
If you are a counsellor, please answer the questions based on the description of the patient.

### Input:
I was with a girl one time and we got back to my place. She said she was going to go to the doctor, and everything seemed resolved, but the police were there and I got arrested. I was charged with kidnapping and some other stuff, and I ended up serving some time.

### Response:
It sounds like you experienced a difficult situation with the girl and the police. It's important to understand what happened and how it affected you. Can you tell me more about the events leading up to your arrest and how it has impacted your life since then? This information can help us better understand your situation and provide appropriate support. Remember, it's okay to share your feelings and experiences. We're here to help.<|en

In [20]:
FastLanguageModel.for_inference(model) # Enabling native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "If you are a counsellor, please answer the questions based on the description of the patient.", # instruction
        "I don't like to talk about this, but I'm afraid that my partner secretly agrees with my belief that unless I'm skinny, I'm not attractive or worthy enough. I think all guys think that way and I believe my partner should love me regardless of my weight.", # input
        "", 
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 350)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
If you are a counsellor, please answer the questions based on the description of the patient.

### Input:
I don't like to talk about this, but I'm afraid that my partner secretly agrees with my belief that unless I'm skinny, I'm not attractive or worthy enough. I think all guys think that way and I believe my partner should love me regardless of my weight.

### Response:
It's understandable that you feel this way, but it's important to remember that your worth and attractiveness are not determined by your weight. It's essential to have open and honest conversations with your partner about your feelings and concerns. Can you share any specific instances where you feel your partner has expressed this belief? This will help us better understand the situation and provide appropriate guidance. Remember, it's okay to h

In [ ]:
final_train_dataset[0]

{'Unnamed: 0': 1702,
 'input': "I got suspended from school for doing drugs and realized I should try to accept that I need help. However, I've been through a lot and feel like I could probably handle more on my own.",
 'output': "It's good that you've acknowledged the need for help, but relying solely on your own ability to handle things might not be the best approach. Seeking professional assistance can provide you with valuable guidance and support, and it's important to remember that reaching out for help is a sign of strength, not weakness. What are your biggest concerns about seeking help?",
 'formatted_data': "<s> ### Input: I got suspended from school for doing drugs and realized I should try to accept that I need help. However, I've been through a lot and feel like I could probably handle more on my own.\n ### Response:  It's good that you've acknowledged the need for help, but relying solely on your own ability to handle things might not be the best approach. Seeking professi

In [ ]:
text = "I started taking the prescribed medication, and after a few sessions, I felt better and more adjusted. As immigrants, my family always worked hard, and I juggled both work and school. But, at times, I feel out of place when I see other students driving to school."
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=500)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I started taking the prescribed medication, and after a few sessions, I felt better and more adjusted. As immigrants, my family always worked hard, and I juggled both work and school. But, at times, I feel out of place when I see other students driving to school. I don't have a car, and I don't have a driver's license. I feel like I'm not as independent as my peers. I also feel like I'm not as good as them in school. I'm not sure if I'm just being too hard on myself or if I'm really not as good as them. I'm not sure if I should be worried about this.
 ### Input:  I understand that you're feeling out of place and comparing yourself to others. It's important to remember that everyone has their own unique experiences and strengths. It's not about being the best, but rather about finding your own path and being true to yourself. If you're worried about your academic performance, you can discuss it with your teachers or a counselor to get a better understanding of your abilities and areas f

In [21]:
trainer.save_model('/content/llama3_finetuned')

In [22]:
from huggingface_hub import HfApi
from google.colab import userdata
api = HfApi()
api.upload_folder(
    folder_path="/content/llama3_finetuned",
    repo_id="PrahmodhRaj/Llama-3_Psychiatrist_Chat",
    repo_type="model",
    use_auth_token = userdata.get("huggingface")
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PrahmodhRaj/Llama-3_Psychiatrist_Chat/commit/3b01986053aafb680b83a4a2b5d844c3006de587', commit_message='Upload folder using huggingface_hub', commit_description='', oid='3b01986053aafb680b83a4a2b5d844c3006de587', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import AutoPeftModelForCausalLM
from peft import LoraConfig
model_id = "PrahmodhRaj/Llama-3_Psychiatrist_Chat"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)